# 0. Define helper functions

In [1]:

## Define data wrangling function for synonymous variants
def train_on_synonymous(ht):
    """
    This function trains the regression model to predict the expected singleton/total variants ratio.
    
    Requirements (things to change as for best parcitces)
    1. Already annotated mutation rates for each tri-nucleotide context in the main hail table
    """
    # Filter only synonymous variants
    ht_syn = ht.filter(ht.vep.most_severe_consequence == "synonymous_variant")

    # Calculate number of variants in each tri-nucleotide context in synonymous variants
    ht_syn_N_variants = (ht_syn.group_by(ht_syn.context, ht_syn.ref, ht_syn.alt, ht_syn.mu_snp, ht_syn.methylation_level).aggregate(N_variants = hl.agg.count()))

    # Calculate number of singletons for each tri-nucleotide context in synonymous variants
    ht_syn_singletons = ht_syn.filter(ht_syn.info.singleton == 1)
    ht_syn_N_singletons = (ht_syn_singletons.group_by(ht_syn_singletons.context, ht_syn_singletons.ref, ht_syn_singletons.alt, ht_syn_singletons.mu_snp, ht_syn_singletons.methylation_level).aggregate(N_singletons = hl.agg.count()))

    # Merge the N variants and N singletons tables
    ht_syn_ps  = ht_syn_N_variants.join(ht_syn_N_singletons, how = 'outer') # outer as all will match and we want all
    ht_syn_ps = ht_syn_ps.annotate(ps = ht_syn_ps.N_singletons/ht_syn_ps.N_variants)
    
    return ht_syn_ps 

## Define regression function
def regress_per_context(ht, ht_syn_lm):
    """
    Calculates MAPS with Standard Error of the Mean for a given consequence.
    
    Requires input in form of Hail table: (so the things that should be changes for easier accessibility):
        1) Mutation rates are to be annotated for each tri-nucleotide context. 
    """
    ht_reg_table = ht.annotate(consequence = ht.vep.most_severe_consequence)

    # Count number of variants and singletons
    ht_reg_table_N_variants = (ht_reg_table.group_by(ht_reg_table.context, ht_reg_table.ref, ht_reg_table.alt, ht_reg_table.mu_snp, ht_reg_table.methylation_level, ht_reg_table.consequence).aggregate(N_variants = hl.agg.count()))
    ht_reg_table_N_singletons = (ht_reg_table.group_by(ht_reg_table.context, ht_reg_table.ref, ht_reg_table.alt, ht_reg_table.mu_snp, ht_reg_table.methylation_level, ht_reg_table.consequence).aggregate(N_singletons = hl.agg.sum(ht_reg_table.info.singleton)))

    # Merge the tables to obtain proportions (ps)
    ht_reg_table_ps = ht_reg_table_N_variants.annotate(**ht_reg_table_N_singletons[ht_reg_table_N_variants.context, ht_reg_table_N_variants.ref, ht_reg_table_N_variants.alt, ht_reg_table_N_variants.mu_snp, ht_reg_table_N_variants.methylation_level, ht_reg_table_N_variants.consequence])
    ht_reg_table_ps = ht_reg_table_ps.annotate(ps = ht_reg_table_ps.N_singletons/ht_reg_table_ps.N_variants)
    
    # Get expected number of singletons by applying the model factors
    ht_reg_table_ps_lm_cons = ht_reg_table_ps.annotate(expected_singletons=(ht_reg_table_ps.mu_snp * ht_syn_lm[1] + ht_syn_lm[0]) * ht_reg_table_ps.N_variants)

    # To aggregate just sum for the context
    ht_reg_table_ps_lm_cons_agg = (ht_reg_table_ps_lm_cons.group_by("consequence")
                  .aggregate(N_singletons=hl.agg.sum(ht_reg_table_ps_lm_cons.N_singletons),
                             expected_singletons=hl.agg.sum(ht_reg_table_ps_lm_cons.expected_singletons),
                             N_variants=hl.agg.sum(ht_reg_table_ps_lm_cons.N_variants)))

    # Calculate MAPS and aggregated proportions 
    ht_reg_table_ps_lm_cons_agg_MAPS = ht_reg_table_ps_lm_cons_agg.annotate(ps_agg=ht_reg_table_ps_lm_cons_agg.N_singletons / ht_reg_table_ps_lm_cons_agg.N_variants,
        maps=(ht_reg_table_ps_lm_cons_agg.N_singletons - ht_reg_table_ps_lm_cons_agg.expected_singletons) / ht_reg_table_ps_lm_cons_agg.N_variants)

    # Add MAPS standard error of the mean (sem)
    ht_reg_table_ps_lm_cons_agg_MAPS = ht_reg_table_ps_lm_cons_agg_MAPS.annotate(maps_sem=(ht_reg_table_ps_lm_cons_agg_MAPS.ps_agg * (1 - ht_reg_table_ps_lm_cons_agg_MAPS.ps_agg) / ht_reg_table_ps_lm_cons_agg_MAPS.N_variants) ** 0.5)
    return ht_reg_table_ps_lm_cons_agg_MAPS

def collapse_strand(ht):
    """
    Return the deduplicated context by collapsing DNA strands.
    
    Function returns the reverse complement if the reference allele is either 'G' or 'T'.
    
    The reverse_complement_bases function has been made obsolete and should be replaced by `hl.reverse_complement`.
    
    :param ht: Input Table.
    :return: Table with deduplicated context annotation (ref, alt, context, was_flipped).
    """
    collapse_expr = {
        'ref': hl.if_else(((ht.ref == 'G') | (ht.ref == 'T')),
                       hl.reverse_complement(ht.ref), ht.ref),
        'alt': hl.if_else(((ht.ref == 'G') | (ht.ref == 'T')),
                       hl.reverse_complement(ht.alt), ht.alt),
        'context': hl.if_else(((ht.ref == 'G') | (ht.ref == 'T')),
                           hl.reverse_complement(ht.context), ht.context),
        'was_flipped': (ht.ref == 'G') | (ht.ref == 'T')
    }
    return ht.annotate(**collapse_expr) if isinstance(ht, hl.Table) else ht.annotate_rows(**collapse_expr)



# 1. Import packages

In [2]:
import hail as hl
from bokeh.io import output_notebook,show
import gnomad.utils.vep
from hail.ggplot import *
import plotly
import plotly.io as pio
pio.renderers.default='iframe'


In [3]:
# 2. Import data

In [4]:
# Import gnomaAD v.3.1.2
ht = hl.read_table('gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht')

#local-run#
ht = ht.head(1000000)
ht = ht.checkpoint('ht_head_100kk.ht')

# Import mutation rates from gnomAD paper
ht_mu = hl.import_table('gs://dataproc-staging-us-central1-629456523523-atyywq8w/supplementary_dataset_10_mutation_rates.tsv.gz',
                delimiter='\t', impute=True, force_bgz=True)
ht_mu = ht_mu.key_by('context', 'ref', 'alt', 'methylation_level') # has to have a key in order to join using foreign key

# Import context table from gnomad (https://broadinstitute.github.io/gnomad_methods/api_reference/utils/vep.html?highlight=context#gnomad.utils.vep.get_vep_context)
context_table = gnomad.utils.vep.get_vep_context("GRCh38").ht()
context_table = context_table.filter(hl.is_defined(context_table.methyl_mean))

context_table_parsed = context_table.select(context_table.context, context_table.methyl_mean)
context_table_parsed = context_table_parsed.transmute(context = context_table_parsed.context[2:5])
context_table_parsed = context_table_parsed.annotate(methyl_mean = hl.float64(context_table_parsed.methyl_mean))
context_table_parsed = context_table_parsed.annotate(methylation_level = hl.case().when(
            context_table_parsed.methyl_mean > 0.6, 2
        ).when(
            context_table_parsed.methyl_mean > 0.2, 1
        ).default(0))

Initializing Hail with default parameters...


2022-10-12 16:14:59 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://192.168.0.181:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /Users/adrian/BroadIS/01_maps/hail-20221012-1614-0.2.99-57537fea08d4.log
2022-10-12 16:19:59 Hail: INFO: wrote table with 1000000 rows in 105 partitions to ht_head_100k.ht
2022-10-12 16:20:03 Hail: INFO: Reading table to impute column types
2022-10-12 16:20:05 Hail: INFO: Finished type imputation
  Loading field 'context' as type str (imputed)
  Loading field 'ref' as type str (imputed)
  Loading field 'alt' as type str (imputed)
  Loading field 'methylation_level' as type int32 (imputed)
  Loading field 'mu_snp' as type float64 (imputed)


In [5]:
# Show the data structure
ht.show(3)

+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:10031    | ["T","C"]  |
| chr1:10037    | ["T","C"]  |
| chr1:10043    | ["T","C"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,56642,0),(2,1.65e-05,121094,0),(0,0.00e+00,25546,0),(0,0.00e... |
| [(2,2.60e-05,76882,0),(4,3.15e-05,126902,0),(0,0.00e+00,34670,0),(1,1.80e... |
| [(1,1.17e-05,85634,0),(1,8.24e-06,121430,0),(0,0.00e+00,39236,0),(0,0.00e... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                          |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [14149,7330,6840,3904,23217,2436,1647,594,176,118,49,29,19,9,11,6,5,2,4,2]   |
| [10121,5441,5610,3838,29394,3741,2868,1281,401,358,163,63,69,31,18,20,10,... |
| [6578,3782,4202,3336,31093,4532,3696,1752,595,585,261,88,82,59,17,26,14,5... |
+------------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_q

In [6]:
# Table with methylation level and mutational rate in the trinucleotide context
ht_mu.show(3)

,,,,
context,ref,alt,methylation_level,mu_snp
str,str,str,int32,float64
"""AAA""","""A""","""C""",0,1.83e-09
"""AAA""","""A""","""G""",0,3.03e-09
"""AAA""","""A""","""T""",0,1.07e-09


In [7]:
# This table contains already precalculated nucleotides -3/+3 from mutation site 
context_table_parsed.show(3)

,,,,
locus,alleles,context,methyl_mean,methylation_level
locus<GRCh38>,array<str>,str,float64,int32
chr1:10469,"[""C"",""A""]","""TCG""",2.54e+01,2
chr1:10469,"[""C"",""G""]","""TCG""",2.54e+01,2
chr1:10469,"[""C"",""T""]","""TCG""",2.54e+01,2


# 3. Add context field to main data

In [8]:
# Before joining the tri-nucleotide context of mutation
ht.count()

1000000

In [9]:
# Split alleles field to ref and alt allele and collapse strands
ht = ht.annotate(ref=ht.alleles[0], alt=ht.alleles[1])
ht = ht.annotate(**context_table_parsed[ht.locus, ht.alleles])
ht = collapse_strand(ht) # Collapse strands as some mutation contexts may not be present in mutation table

In [10]:
# (SNP can be represented as both strands so collapsing makes the schema uniformal) 
ht.count()

1000000

In [11]:
ht.show(3)

+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:10031    | ["T","C"]  |
| chr1:10037    | ["T","C"]  |
| chr1:10043    | ["T","C"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,56642,0),(2,1.65e-05,121094,0),(0,0.00e+00,25546,0),(0,0.00e... |
| [(2,2.60e-05,76882,0),(4,3.15e-05,126902,0),(0,0.00e+00,34670,0),(1,1.80e... |
| [(1,1.17e-05,85634,0),(1,8.24e-06,121430,0),(0,0.00e+00,39236,0),(0,0.00e... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                          |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [14149,7330,6840,3904,23217,2436,1647,594,176,118,49,29,19,9,11,6,5,2,4,2]   |
| [10121,5441,5610,3838,29394,3741,2868,1281,401,358,163,63,69,31,18,20,10,... |
| [6578,3782,4202,3336,31093,4532,3696,1752,595,585,261,88,82,59,17,26,14,5... |
+------------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_q

In [12]:
ht = ht.filter(hl.is_defined(ht.methylation_level)) # Filter to only variants with calculated methylation metric

# 4. Add mutation rates for added contexts

In [13]:
# Add mutation rates according to the context, methylation level but also ref and alt allele for this context
ht = ht.annotate(**ht_mu[ht.context, ht.ref, ht.alt, ht.methylation_level])
#local-run# ht.checkpoint('ht_per_variant.ht')

# 5. Train linear model on synonymous variants for mutational class correction

In [14]:
ht_syn_ps = train_on_synonymous(ht)

2022-10-12 16:20:26 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'freq_meta' -> 'freq_meta_1'
    'filtering_model' -> 'filtering_model_1'
    'vep_csq_header' -> 'vep_csq_header_1'
    'freq_sample_count' -> 'freq_sample_count_1'
    'dbsnp_version' -> 'dbsnp_version_1'
    'age_distribution' -> 'age_distribution_1'
    'faf_index_dict' -> 'faf_index_dict_1'
    'faf_meta' -> 'faf_meta_1'
    'freq_index_dict' -> 'freq_index_dict_1'
    'vep_version' -> 'vep_version_1'


In [15]:
ht_syn_ps.show()

2022-10-12 16:24:16 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 16:24:18 Hail: INFO: Coerced sorted dataset              (0 + 1) / 1]
2022-10-12 16:27:47 Hail: INFO: Coerced sorted dataset>           (16 + 4) / 20]
2022-10-12 16:31:17 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 16:31:18 Hail: INFO: Coerced sorted dataset
2022-10-12 16:34:40 Hail: INFO: Coerced sorted dataset===>        (17 + 3) / 20]


,,,,,,,
context,ref,alt,mu_snp,methylation_level,N_variants,N_singletons,ps
str,str,str,float64,int32,int64,int64,float64
"""ACG""","""C""","""A""",NA,1,3,NA,NA
"""ACG""","""C""","""A""",NA,2,29,NA,NA
"""ACG""","""C""","""A""",NA,1,NA,2,NA
"""ACG""","""C""","""A""",NA,2,NA,12,NA
"""ACG""","""C""","""G""",4.94e-09,0,3,2,6.67e-01
"""ACG""","""C""","""G""",NA,1,1,NA,NA
"""ACG""","""C""","""G""",NA,2,24,NA,NA
"""ACG""","""C""","""G""",NA,2,NA,12,NA


In [16]:
ht_syn_ps.count()

2022-10-12 16:38:15 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 16:38:16 Hail: INFO: Coerced sorted dataset
2022-10-12 16:41:41 Hail: INFO: Coerced sorted dataset=========>  (19 + 1) / 20]
2022-10-12 16:45:03 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 16:45:04 Hail: INFO: Coerced sorted dataset              (0 + 1) / 1]
2022-10-12 16:48:27 Hail: INFO: Coerced sorted dataset>           (16 + 4) / 20]


49

In [17]:
# Perform regression
ht_syn_lm = ht_syn_ps.aggregate(hl.agg.linreg(ht_syn_ps.ps, [1, ht_syn_ps.mu_snp], weight=ht_syn_ps.N_variants).beta)
# Show intercept and beta
ht_syn_lm

2022-10-12 16:51:55 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 16:51:56 Hail: INFO: Coerced sorted dataset
2022-10-12 16:55:24 Hail: INFO: Coerced sorted dataset>           (16 + 4) / 20]
2022-10-12 16:59:01 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 16:59:03 Hail: INFO: Coerced sorted dataset              (0 + 1) / 1]
2022-10-12 17:02:30 Hail: INFO: Coerced sorted dataset=========>  (19 + 1) / 20]


[0.5107269978602446, -2569184.294925339]

# 6. Predict expected number of variants for each context

In [18]:
ht_syn_lm

[0.5107269978602446, -2569184.294925339]

In [19]:
maps_table = regress_per_context(ht, ht_syn_lm)
#local-run# maps_table.checkpoint('maps_table_per_variant.ht')

In [20]:
maps_table.show(maps_table.count())

2022-10-12 17:05:58 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 17:05:59 Hail: INFO: Coerced sorted dataset
2022-10-12 17:09:18 Hail: INFO: Coerced sorted dataset======>     (18 + 2) / 20]
2022-10-12 17:09:20 Hail: INFO: Ordering unsorted dataset with network shuffle0]
2022-10-12 17:13:00 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 17:13:01 Hail: INFO: Coerced sorted dataset
2022-10-12 17:16:26 Hail: INFO: Coerced sorted dataset>           (16 + 4) / 20]
2022-10-12 17:19:56 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 17:19:57 Hail: INFO: Coerced sorted dataset              (0 + 1) / 1]
2022-10-12 17:23:28 Hail: INFO: Coerced sorted dataset=========>  (19 + 1) / 20]
2022-10-12 17:23:32 Hail: INFO: Ordering unsorted dataset with network shuffle0]


,,,,,,
consequence,N_singletons,expected_singletons,N_variants,ps_agg,maps,maps_sem
str,int64,float64,int64,float64,float64,float64
"""3_prime_UTR_variant""",851,9.83e+02,6337,1.34e-01,-2.08e-02,4.28e-03
"""5_prime_UTR_variant""",1248,1.13e+03,5717,2.18e-01,2.07e-02,5.46e-03
"""TF_binding_site_variant""",25,2.22e+01,146,1.71e-01,1.90e-02,3.12e-02
"""downstream_gene_variant""",509,7.49e+02,4781,1.06e-01,-5.01e-02,4.46e-03
"""intergenic_variant""",896,1.46e+03,9454,9.48e-02,-5.92e-02,3.01e-03
"""intron_variant""",9526,1.33e+04,84152,1.13e-01,-4.46e-02,1.09e-03
"""mature_miRNA_variant""",5,4.65e+00,28,1.79e-01,1.26e-02,7.24e-02
"""missense_variant""",1705,1.47e+03,8679,1.96e-01,2.73e-02,4.26e-03


In [21]:
ggplot(maps_table, aes(x=maps_table.consequence, y = maps_table.maps)) + geom_col(aes(fill=maps_table.consequence))

2022-10-12 17:27:17 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 17:27:18 Hail: INFO: Coerced sorted dataset
2022-10-12 17:31:13 Hail: INFO: Coerced sorted dataset===>        (17 + 3) / 20]
2022-10-12 17:34:50 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 17:34:51 Hail: INFO: Coerced sorted dataset
2022-10-12 17:38:27 Hail: INFO: Coerced sorted dataset=========>  (19 + 1) / 20]
2022-10-12 17:38:31 Hail: INFO: Ordering unsorted dataset with network shuffle0]


In [22]:
#7. Compute number of variants in each class

In [23]:
ht_singletons = ht.filter(ht.info.singleton == 1)
N_singletons = ht_singletons.aggregate(hl.agg.counter(ht_singletons.vep.most_severe_consequence))

N_variants = ht.aggregate(hl.agg.counter(ht.vep.most_severe_consequence))

In [25]:
N_singletons

frozendict({'3_prime_UTR_variant': 1552, '5_prime_UTR_variant': 1931, 'TF_binding_site_variant': 42, 'downstream_gene_variant': 992, 'intergenic_variant': 1906, 'intron_variant': 18056, 'mature_miRNA_variant': 8, 'missense_variant': 2724, 'non_coding_transcript_exon_variant': 3558, 'regulatory_region_variant': 309, 'splice_acceptor_variant': 3, 'splice_donor_variant': 22, 'splice_region_variant': 268, 'stop_gained': 107, 'synonymous_variant': 1181, 'upstream_gene_variant': 3443})

In [26]:
N_variants

frozendict({'3_prime_UTR_variant': 6337, '5_prime_UTR_variant': 5717, 'TF_binding_site_variant': 146, 'downstream_gene_variant': 4781, 'intergenic_variant': 9454, 'intron_variant': 84152, 'mature_miRNA_variant': 28, 'missense_variant': 8679, 'non_coding_transcript_exon_variant': 15516, 'regulatory_region_variant': 1346, 'splice_acceptor_variant': 3, 'splice_donor_variant': 56, 'splice_region_variant': 999, 'stop_gained': 254, 'synonymous_variant': 4334, 'upstream_gene_variant': 12636})

In [27]:
# 8. Export tables for plotting and exploring
ht.write('ht_final.ht')
N_singletons.export('N_singletons.csv', delimiter=',')
N_variants.export('N_variants.csv', delimiter=',')
maps_table.export('maps_table.csv', delimiter=',')

2022-10-12 18:32:16 Hail: INFO: Ordering unsorted dataset with network shuffle5]
2022-10-12 18:32:17 Hail: INFO: Coerced sorted dataset
2022-10-12 18:37:18 Hail: INFO: Ordering unsorted dataset with network shuffle0]
2022-10-12 18:37:40 Hail: INFO: wrote table with 154438 rows in 20 partitions to ht_final.ht


AttributeError: 'frozendict' object has no attribute 'export'